## There is no concept of built-in

>Julia is designed to be easy and fast and questions notions generally held to
>be “laws of nature” by practitioners of numerical computing:
>1. High-level dynamic programs have to be slow.
>2. One must prototype in one language and then rewrite in another language for speed or deployment.
>3. There are parts of a system appropriate for the programmer, and other parts that are best left untouched as they have been built by the experts.

Bezanson, Jeff, et al. "Julia: A fresh approach to numerical computing." arXiv preprint arXiv:1411.1607 (2014).


## User-defined functions are not slow

In [19]:
using Random, LinearAlgebra
# Your own functions are not slow
function my_sum(x)
    s = zero(eltype(x))
    for ix in x
        s+=ix
    end
    s
end

my_sum (generic function with 1 method)

In [21]:
a = rand(1000000);
@time my_sum(a)

  0.005145 seconds (5 allocations: 176 bytes)


499537.6819078561

In [22]:
a = rand(100)
@code_warntype my_sum(a)

Body::Float64
│╻╷╷  iterate5 1 ── %1  = (Base.arraylen)(x)::Int64
││╻╷   iterate  │    %2  = (Base.sle_int)(0, %1)::Bool
│││╻    <  │    %3  = (Base.bitcast)(UInt64, %1)::UInt64
││││╻    <  │    %4  = (Base.ult_int)(0x0000000000000000, %3)::Bool
││││╻    &  │    %5  = (Base.and_int)(%2, %4)::Bool
│││    └───       goto #3 if not %5
│││╻    getindex  2 ── %7  = (Base.arrayref)(false, x, 1)::Float64
│││    └───       goto #4
│││    3 ──       goto #4
││     4 ┄─ %10 = φ (#2 => false, #3 => true)::Bool
││     │    %11 = φ (#2 => %7)::Float64
││     │    %12 = φ (#2 => 2)::Int64
││     └───       goto #5
│      5 ── %14 = (Base.not_int)(%10)::Bool
│      └───       goto #11 if not %14
│      6 ┄─ %16 = φ (#5 => 0.0, #10 => %19)::Float64
│      │    %17 = φ (#5 => %11, #10 => %33)::Float64
│      │    %18 = φ (#5 => %12, #10 => %34)::Int64
│╻    +6 │    %19 = (Base.add_float)(%16, %17)::Float64
││╻    rem  │    %20 = (Base.bitcast)(UInt64, %18)::UInt64
││╻    -  │    %21 = π (0x000000000000

### Function works on any input type (JIT-compilation)

In [ ]:
my_sum(x_int)

## User-defined types are not slow - and magic

In [23]:
struct ModInt{n} <: Integer
    k::Int
    ModInt{n}(k) where n =new(mod(k,n))
end

Base.show(io::IO, k::ModInt{n}) where n =
    print(io, get(io, :compact, false) ? k.k : "$(k.k) mod $n")

import Base: +, -, *, /, inv, <, show, oneunit

+(a::ModInt{n}, b::ModInt{n}) where n = ModInt{n}(a.k+b.k)
-(a::ModInt{n}, b::ModInt{n}) where n = ModInt{n}(a.k-b.k)
*(a::ModInt{n}, b::ModInt{n}) where n = ModInt{n}(a.k*b.k)
-(a::ModInt{n}) where n = ModInt{n}(-a.k)

inv(a::ModInt{n}) where n = ModInt{n}(invmod(a.k, n))
/(a::ModInt, b::ModInt) = a*inv(b)
oneunit(T::Type{ModInt{n}}) where n = ModInt{n}(1)

Base.convert(::Type{ModInt{n}}, i::Int) where n = ModInt{n}(i)
Base.promote_rule(::Type{ModInt{n}}, ::Type{Int}) where n = ModInt{n}

Test the functionality we just defined

In [26]:
a = ModInt{5}(8)

3 mod 5

### Arrays of ModInts

* matrix multiplication
* diagm
* performance

In [29]:
x = ModInt{5}.(rand(Int,4))
y = ModInt{5}.(rand(Int,4))
x + y

4-element Array{ModInt{5},1}:
 0 mod 5
 1 mod 5
 1 mod 5
 4 mod 5

In [32]:
const M5 = ModInt{5}
x = M5.(rand(Int,3,3))
y = M5.(rand(Int,3,5))
x*y

3×5 Array{ModInt{5},2}:
 1  0  1  4  3
 3  4  4  1  2
 2  1  0  4  2

In [33]:
ones(M5,10,10)

10×10 Array{ModInt{5},2}:
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1

In [36]:
diagm(1=>ones(M5,9))

10×10 Array{ModInt{5},2}:
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0

This is still fast, the following operations involves 10 million Modint multiplications.

### Exercise

1. Write a function that counts the number of elements in an array that are less than a user-defined threshold

In [37]:
# Task 1
function f(x,tres)
    s = 0
    for ix in x
        if ix < tres
            s = s+1
        end
    end
    s
end

f (generic function with 1 method)

In [38]:
using Test

@test f([0.05,0.8,2.0,0.06],0.1) == 2
@test f(collect(1:10),3.5) == 3

Test Passed

2. Write a function that calculates the sum of squares of an array

In [41]:
# Task 2
function g(x)
    s = zero(eltype(x))
    for ix in x
        s += ix^2
    end
    s
end
# Define a function g here

g (generic function with 1 method)

In [51]:
@test g([2.0, 3.5, 1.5]) == 18.5
@test g([4,5,6]) == 77

Test Passed